In [146]:
import csv
import collections
import numpy as np
from datetime import datetime, timedelta
from scipy import stats

In [102]:
event_data = []
dates = set([])
with open("/Users/charlesvidrine/data/events_US.csv", 'rU') as events:
    reader = csv.reader(events)
    reader.next()
    for row in reader:
        date = row[0]
        elems = date.split("/")
        date_ = ""
        for elem in elems:
            if len(elem) == 1:
                elem = '0' + elem
            date_ = date_ + elem + "/"
        date_ = date_[:len(date_) - 1]
        if len(date_) == 8:
            try:
                date_tuple = tuple([datetime.strptime(date_, "%d/%m/%y"), row[1]])
                dates.add(date_tuple)
            except:
                continue
#                 dates.add(datetime.strptime(date_, "%d/%m/%y") - timedelta(days=1))
# #                 dates.add(datetime.strptime(date_, "%d/%m/%y") + timedelta(days=1))
        else:
            try:
                date_tuple = tuple([datetime.strptime(date_, "%d/%m/%y"), row[1]])
                dates.add(date_tuple)
            except:
                continue
#                 dates.add(datetime.strptime(date_, "%d/%m/%Y") - timedelta(days=1))
#                 dates.add(datetime.strptime(date_, "%d/%m/%Y") + timedelta(days=1))
            
    

In [141]:
airport_id, city, state = 0, 2, 3
id_to_city = collections.defaultdict(str)
with open('/Users/charlesvidrine/data/airports.csv', 'rU') as airports:
    reader = csv.reader(airports)
    headers = next(reader)
    for row in reader:
        id_to_city[row[airport_id]] = row[city]
        
def GetCity(airport_code):
    return id_to_city[airport_code]

airports = ["MDW", "JFK", "LGA", "EWR", "LAX", "ONT", "SNA", "BUR", "ORD", "MDW", "DAL", "DFW", "IAH", "HOU", "IAD", "DCA", "BWI", "MIA", "FLL", "PBI", "PHL", "ATL", "BOS"]
delay_map = collections.defaultdict(lambda: collections.defaultdict(list))
with open("/Users/charlesvidrine/data/elapsed_df.csv") as flight_traffic:
    reader = csv.reader(flight_traffic)
    for row in reader:
        try:
            float(row[25])
        except ValueError:
            continue
        if len(row[2]) == 1:
            row[2] = '0' + row[2]
        if len(row[1]) == 1:
            row[1] = '0' + row[1]
        date = datetime.strptime(row[3] + "/" + row[2] + "/" + row[1], "%d/%m/%Y")
        if len(row[25]) != 0 and not (((date, GetCity(row[5])) in dates) or (date, GetCity(row[6])) in dates):
            delay_map[row[4]][date].append(float(row[25]))
            
    

In [142]:
delay_map_means1 = collections.defaultdict(lambda: collections.defaultdict(float))
for key in delay_map:
    for list_ in delay_map[key]:
        delay_map_means1[key][list_] = np.mean(delay_map[key][list_])
        
        

In [157]:
no_event_data = []
delay_map_means2 = collections.defaultdict(list)
for key in delay_map:
    print key
    total=0
    for list_ in delay_map[key]:
        total+=len(delay_map[key][list_])
        delay_map_means2[key].append(np.mean(delay_map[key][list_]))
    print total
        
delay_airline_means = collections.defaultdict(int)
for key in delay_map_means2:
    event_data.append(delay_map_means2[key])
    delay_airline_means[key] = np.std(delay_map_means2[key])

AA
848021
OO
678201
DL
869799
HA
79189
F9
99128
AS
179136
B6
268075
NK
144867
WN
1269667
VX
63769
EV
319809
UA
558844


In [125]:
delay_airline_means #With Event

defaultdict(int,
            {'AA': 10.824098629072505,
             'AS': 3.71687887005535,
             'B6': 20.447888182139184,
             'DL': 16.06219298127549,
             'EV': 19.895202086839802,
             'F9': 17.207652858525126,
             'HA': 4.334985847485847,
             'NK': 14.613684466190458,
             'OO': 14.90492037743475,
             'UA': 12.19731234051152,
             'VX': 14.373037385162824,
             'WN': 12.848794460033215})

In [129]:
delay_airline_means #With no Event

defaultdict(int,
            {'AA': 7.908411386822841,
             'AS': 2.8821115983589203,
             'B6': 16.338838284841778,
             'DL': 8.55902836870599,
             'EV': 11.7949420542732,
             'F9': 12.768130012069562,
             'HA': 1.3904215276970038,
             'NK': 10.539616451347877,
             'OO': 10.086181758146353,
             'UA': 9.41524249206131,
             'VX': 13.542160704470936,
             'WN': 9.996619970952503})

In [131]:
delay_airline_means #with no event

defaultdict(int,
            {'AA': 6.281113235644872,
             'AS': 7.150947449109138,
             'B6': 15.958093291324754,
             'DL': 13.819772648390602,
             'EV': 9.873686503073431,
             'F9': 9.883501284212006,
             'HA': 6.385802049539686,
             'NK': 9.924169315365017,
             'OO': 7.34157179448326,
             'UA': 8.024145659497782,
             'VX': 14.99249478656366,
             'WN': 6.459520422450286})

In [135]:
delay_airline_means #with event

defaultdict(int,
            {'AA': 11.18317997029387,
             'AS': 9.831432298540705,
             'B6': 23.29898370658462,
             'DL': 25.55105869814888,
             'EV': 26.28067085388276,
             'F9': 27.55925615800656,
             'HA': 12.631012594578474,
             'NK': 21.628616579716912,
             'OO': 30.861637285978176,
             'UA': 18.564659214619276,
             'VX': 19.287273413573754,
             'WN': 11.955909402919682})

In [155]:
for key in delay_airline_means:
    print key, len(delay_map_means1[key])

AA 365
OO 365
DL 365
NK 365
HA 365
F9 365
AS 365
B6 365
WN 365
VX 365
EV 365
UA 365
